An ipywidget interface to move incorrect predictions within the validation dataset should they be labeled wrong

In [1]:
import sys

sys.path.insert(0, "..")
import itertools


import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import torch

import cocpit

%load_ext autoreload
%autoreload 2

### check classifications from specific model and validation dataloader

In [4]:
#  Make predictions on each batch of images,
#  concatenate the precitions across batches,
#  and return the wrong predictions.
#  Loops over folds
p = cocpit.predictions.LoaderPredictions()
p.predict(top_k_preds=8, folds=1)  # the top k predictions will be displayed in bar chart
p.concatenate_loader_vars()
p.find_wrong_indices()

print("DONE FINDING INCORRECT PREDICTIONS!")                
print(f"There are {len(p.all_labels)} total images in the dataloader.")
print(f"There are {len(p.all_labels[p.wrong_idx])} wrong predictions to check in the dataloader!")


DONE FINDING INCORRECT PREDICTIONS!
There are 4308 total images in the dataloader.
There are 73 wrong predictions to check in the dataloader!


In [4]:
# to only look between specific categories run this cell
label_list = dict(zip(cocpit.config.CLASS_NAMES, np.arange(0, len(cocpit.config.CLASS_NAMES))))

# change these two lines to focus on wrong predictions from a specific category
# or box within the confusion matrix
# makes labeling easier focusing on two at a time
human_label = label_list ["budding rosette"]
model_label = label_list["aggregate"]

p.hone_incorrect_predictions(label_list, human_label, model_label)

2 wrong predictions between budding rosette and aggregate


In [5]:
"""
Display a bar chart of model prediction probabilities
A dropdown menu is available to move the image to a different category in the training dataset
If you don't want to move the image and the human labeled correctly, simply click "Next"
"""
display_gui = cocpit.gui_wrong.GUI(p.wrong_trunc, p.all_labels, p.all_paths, p.all_topk_probs, p.all_topk_classes)
display_gui.button_functions()
display(ipywidgets.HBox([display_gui.center, display_gui.menu, display_gui.next_btn]))

In [13]:
# clear out the incorrect predictions between a specific scenario for human and model label.
# If this cell is run after all images have been iterated through, run the second 
# to last cell again to hone in on a different set of human and model predictions
p.wrong_trunc=[]